<a href="https://colab.research.google.com/github/sim017/OCR/blob/master/OCR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np


In [0]:
image_size = 28
no_of_labels = 10
image_pixels = 28*28
train_data = np.loadtxt("mnist_train.csv",delimiter=",")
test_data = np.loadtxt("mnist_test.csv",delimiter=",")
print(test_data)
print(test_data.shape)

In [0]:
#scaling
fac =0.99/255 #to bring every value between the range [0.01,1]
train_imgs = np.array(train_data[:,1:])*fac+0.01
test_imgs = np.array(test_data[:,1:])*fac+0.01
print(test_imgs)
print(test_imgs.shape)
train_labels = np.asfarray(train_data[:, :1])
test_labels = np.asfarray(test_data[:, :1])
print(test_labels)
print(test_labels.shape)

In [0]:
lr = np.arange(10)

for label in range(10):
    one_hot = (lr==label).astype(np.int)
    print("label: ", label, " in one-hot representation: ", one_hot)
lr = np.arange(10)

# transform labels into one hot representation
train_labels_one_hot = (lr==train_labels).astype(np.float)
test_labels_one_hot = (lr==test_labels).astype(np.float)

# we don't want zeroes and ones in the labels neither:
train_labels_one_hot[train_labels_one_hot==0] = 0.01
train_labels_one_hot[train_labels_one_hot==1] = 0.99
test_labels_one_hot[test_labels_one_hot==0] = 0.01
test_labels_one_hot[test_labels_one_hot==1] = 0.99

print(test_labels_one_hot.shape)
print(train_labels_one_hot.shape)

In [0]:
@np.vectorize
def sigmoid(x):
    return 1 / (1 + np.e ** -x)


class NeuralNetwork:
    def __init__(self,no_input_nodes, no_output_nodes, no_hidden_layers):
        self.no_input_nodes = no_input_nodes
        self.no_output_nodes = no_output_nodes
        self.no_hidden_layers = no_hidden_layers
        self.w1 = np.random.randn(self.no_hidden_layers,self.no_input_nodes) #wieght for the hidden layers
        self.b1 = np.zeros(shape=(self.no_hidden_layers,1))
        self.w2 = np.random.randn(self.no_output_nodes,self.no_hidden_layers)
        self.b2 = np.zeros(shape=(self.no_output_nodes,1))
        self.learning_rate = 0.1
        
        self.sigmoid_function = sigmoid
        
      
    
    def forward_backward_propogation(self,input_vector,output_vector,no_of_iterations):
        global train_imgs
        input_vector = np.array(input_vector,ndmin=2).T
        output_vector = np.array(output_vector,ndmin=2).T
        print("input_vector")
        print(input_vector.shape)
        print("output_vector")
        print(output_vector.shape)
        for i in range(0,no_of_iterations):
            output_vector1 = np.dot(self.w1,input_vector) + self.b1
            print("output_vector1")
            print(output_vector1.shape)
            
            output_hidden = self.sigmoid_function(output_vector1)
            print("output_hidden")
            print(output_hidden.shape)
            output_vector2 = np.dot(self.w2,output_hidden)+self.b2
            print("output_vector2")
            print(output_vector2)
            final_output = self.sigmoid_function(output_vector2)
            print("final output")
            print(final_output.shape)
            #define cost function
            m = train_imgs.shape[0]
            logprobs = np.multiply(np.log(final_output),output_vector)+np.multiply((1-output_vector),np.log(1-final_output))
            cost = -np.sum(logprobs)/m
            dz2 = final_output - output_vector
            dw2 = (1/m)*(np.dot(dz2,output_hidden.T))
            db2 = (1/m)*(np.sum(dz2,axis=1,keepdims=True))
            dz1 = np.multiply(np.dot(self.w2.T,dz2),1-np.power(output_hidden,2))
            dw1 = (1/m)*np.dot(dz1,input_vector.T)
            db1 = (1/m)*np.sum(dz1,axis=1,keepdims=True)

            self.w1 =self.w1 -self.learning_rate*dw1
            self.w2 = self.w2 -self.learning_rate*dw2
            self.b1 = self.b1 - self.learning_rate*db1
            self.b2 = self.b2 - self.learning_rate*db2
            
    def forward_propogation_predict(self,input_vector):
        input_vector = np.array(input_vector,ndmin=2).T
#         output_vector = np.array(output_vector,ndmin=2).T
        output_vector1 = np.dot(self.w1,input_vector) + self.b1
        output_hidden = self.sigmoid_function(output_vector1)
        output_vector2 = np.dot(self.w2,output_hidden)+self.b2
        final_output = self.sigmoid_function(output_vector2)
        return final_output
    def predict_accuracy(self, input_vector, output_vector,no_of_iterations,test_input_vector,test_output_vector):
        global test_labels
        self.forward_backward_propogation(input_vector,output_vector,no_of_iterations)
        Y = self.forward_propogation_predict(test_input_vector)
        print(Y)
        print(Y[:][0].shape)
        print(Y.shape)
        for i in range(0,20):
            print(test_labels[i],np.argmax(Y[:][i]),np.max(Y[i]))
            
        accuracy = float((np.dot(test_output_vector,Y)+np.dot(1-test_output_vector,1-Y))/float(test_output_vector.size)*100)
        print("Accuracy {} %",accuracy)

In [0]:
NeuralNetwork(image_pixels,10,100).predict_accuracy(train_imgs,train_labels_one_hot,1,test_imgs,test_labels_one_hot)